In [ ]:
# Setup the notebook based on running environment.
import os
# Check for kaggle environment.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-ai-generativelanguage pydrive2 tensorflow tensorflow-decision-forests cryptography pyOpenSSL langchain langchain-core nltk ray click google-generativeai google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway nltk preprocessing gradio dopamine-rl torchtune
    !pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    !pip install -qU langchain-community langchain-text-splitters google-adk google-adk[eval] google-cloud-translate Pillow
    from kaggle_secrets import UserSecretsClient # type: ignore
else:
    # Mock the kaggle secrets client.
    class UserSecretsClient:
        @classmethod
        def set_secret(cls, id: str, value: str):
            os.environ[id] = value
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    %pip install -qU langchain-community langchain-text-splitters pandas google-api-core ollama google-adk "google-adk[eval]" Pillow

import asyncio, io, IPython, time
from google import genai
from google.api_core import retry, exceptions
from google.genai.models import Models
from google.genai import types, errors
from io import BytesIO
from PIL import Image

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

is_retriable = lambda e: (isinstance(e, errors.APIError) and e.code in {429, 503, 500})
Models.generate_images = retry.Retry(predicate=is_retriable)(Models.generate_images)
Models.generate_videos = retry.Retry(predicate=is_retriable)(Models.generate_videos)
Models.generate_content = retry.Retry(predicate=is_retriable)(Models.generate_content)

In [ ]:
img_in = Image.open("docs/src/vales_of_anduin_sketch.jpg")

prompt = """Do something cool with this image to show off your skills."""

In [ ]:
# Cannot generate images or video.
response = client.models.generate_content(
    model="gemini-3-pro-preview",
    contents=prompt,
)

print(response.text)

In [ ]:
# Capable of editing and generating.
response = client.models.generate_content(
    model="gemini-3-pro-image-preview",
    contents=prompt,
)

print(response.text)

In [ ]:
# Can only generate, no editing.
def generate_image(prompt):
    result = client.models.generate_images(
        model="imagen-4.0-ultra-generate-001", # imagen-4.0-generate-001, imagen-4.0-fast-generate-001
        prompt=prompt,
        config=dict(aspect_ratio="16:9", image_size="2k") # person_generation="DONT_ALLOW"
    )
    for n, generated_image in enumerate(result.generated_images):
        generated_image.image.save(f"docs/results/scene_{n}.jpg")
        generated_image.image.show()

# Capable of editing and generating.
def generate_video(image, prompt):
    # Converting the image to bytes
    image_bytes_io = io.BytesIO()
    image.save(image_bytes_io, format=image.format)
    image_bytes = image_bytes_io.getvalue()

    operation = client.models.generate_videos(
        model="veo-3.0-generate-001", # veo-3.0-generate-001, veo-3.0-fast-generate-001
        prompt=prompt,
        image=types.Image(image_bytes=image_bytes, mime_type=image.format),
        config=types.GenerateVideosConfig(
            aspect_ratio="16:9",
            number_of_videos=1))
    
    while not operation.done:
        print("Waiting for video generation to complete...")
        time.sleep(10)
        operation = client.operations.get(operation)

    return operation.result.generated_videos[0]

In [ ]:
# Capable of editing and generating.
result = client.models.generate_content(
    model="gemini-2.5-flash-image", # gemini-2.5-flash-image-preview
    contents=[prompt, img_in]
)

for part in result.candidates[0].content.parts:
    if part.text:
        print(part.text)
    elif part.inline_data is not None:
        image = Image.open(BytesIO(part.inline_data.data))
        image.save("docs/results/img_out.png")

Image.open("docs/results/img_out.png")

In [ ]:
lyria_client = genai.Client(
    api_key=GOOGLE_API_KEY,
    http_options={'api_version': 'v1alpha'}, # Lyria is only experimental.
)

music_prompts = [
    {"text": "Piano", "weight": 2.0},
    types.WeightedPrompt(text="Meditation", weight=0.5),
    types.WeightedPrompt(text="Live Performance", weight=1.0),
]

music_config = types.LiveMusicGenerationConfig(
    bpm=128,
    scale=types.Scale.D_MAJOR_B_MINOR,
    music_generation_mode=types.MusicGenerationMode.QUALITY
)

async def generate_music(prompts: list, config: types.LiveMusicGenerationConfig):
    # Background task to process incoming audio.
    async def receive_audio(session):
        while True:
            async for message in session.receive():
                audio_data = message.server_content.audio_chunks[0].data
                # Process audio...
                await asyncio.sleep(10**-12)

    async with (
        lyria_client.aio.live.music.connect(model='models/lyria-realtime-exp') as session,
        asyncio.TaskGroup() as tg,
    ):
        # Set up task to receive server messages.
        tg.create_task(receive_audio(session))

        # Send initial prompts and config
        await session.set_weighted_prompts(prompts=music_prompts)
        await session.set_music_generation_config(config=config)

        # Start streaming music
        await session.play()

In [ ]:
asyncio.run(generate_music(music_prompts, music_config))